In [ ]:
!pip install transformers streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.4 MB/s eta 0:00:00


In [ ]:
!pip install accelerate
!pip install pyngrok
!ngrok authtoken 2mOqhQcSnbbYhkfCMidP4Mqrx8e_21rYHbQprP4HVABgcXhky


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
%%writefile app.py
import streamlit as st
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import re

def preprocess_text(text):
    # Bước 1: Làm sạch
    text = text.replace("\n", " ")
    text = re.sub(r"[i|r|k|d]\)", "-", text)
    text = re.sub(r"ng\.\.\.", "v.v.", text)
    text = " ".join(text.split())

    # Bước 2: Tái cấu trúc
    text = text.replace("1.", "1. ").replace("2.", "2. ")
    text = text.replace(";", "; ").replace("-", "- ")

    return text


# -----------------------------------
# Cấu hình ứng dụng
# -----------------------------------

HF_MODEL_NAME = "Lusic/testmodel"  # Tên mô hình trên HuggingFace
MAX_NEW_TOKENS = 256              # Số lượng token tối đa sinh ra
TEMPERATURE = 1.5                 # Độ ngẫu nhiên trong việc sinh văn bản
TOP_P = 0.9                       # Giá trị xác suất tích lũy (top-p sampling)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# -----------------------------------
# Khởi tạo mô hình và tokenizer
# -----------------------------------

@st.cache_resource
def initialize_model_and_tokenizer(model_name):
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(model_name)
        model.to(DEVICE)
        model.eval()
        return tokenizer, model

    except Exception as e:
        st.error(f"Lỗi khi tải mô hình hoặc tokenizer: {e}")
        st.stop()

# Khởi tạo tokenizer và model
tokenizer, model = initialize_model_and_tokenizer(HF_MODEL_NAME)

# -----------------------------------
# Giao diện Streamlit
# -----------------------------------

st.title("🤖 Ứng Dụng Chat với Mô Hình Ngôn Ngữ")

st.markdown(
    """
    Chào mừng bạn đến với ứng dụng Chat AI! Ứng dụng này cho phép bạn tương tác với
    mô hình ngôn ngữ **Lusic/testmodel** từ HuggingFace. Hãy nhập tin nhắn bên dưới và nhận phản hồi từ AI.
    """
)

user_input = st.text_area("📝 Tin nhắn của bạn:", height=150, placeholder="Nhập nội dung tin nhắn tại đây...")
if st.button("💬 Tạo phản hồi"):
    if user_input.strip() == "":
        st.warning("Vui lòng nhập tin nhắn để nhận phản hồi.")
    else:
        with st.spinner("🔄 Đang sinh phản hồi..."):
            try:
                inputs = tokenizer(user_input, return_tensors="pt", truncation=True).to(DEVICE)
                outputs = model.generate(
                    inputs.input_ids,
                    max_new_tokens=MAX_NEW_TOKENS,
                    temperature=TEMPERATURE,
                    top_p=TOP_P,
                )
                response = tokenizer.decode(outputs[0], skip_special_tokens=True)
                st.success("🗨️ **Phản hồi từ mô hình:**")
                st.write(preprocess_text(response))
            except Exception as e:
                st.error(f"Lỗi khi tạo phản hồi: {e}")

st.sidebar.header("📖 Hướng dẫn sử dụng")
st.sidebar.info(
    """
- **Nhập tin nhắn của bạn:** Gõ nội dung tin nhắn vào ô văn bản.
- **Tạo phản hồi:** Nhấn nút "Tạo phản hồi" để nhận câu trả lời từ mô hình.
- **Chi tiết mô hình:** Ứng dụng sử dụng mô hình HuggingFace [Lusic/testmodel](https://huggingface.co/Lusic/testmodel).
- **Yêu cầu hệ thống:** Khuyến nghị chạy trên máy tính có GPU để đạt hiệu suất cao nhất.
- **Lưu ý:** Đảm bảo các thư viện cần thiết đã được cài đặt.
    """
)

st.markdown(
    """
    ---
    🛠️ **Phát triển bởi Streamlit** | 📅 **Ngày:** 8 Tháng 12, 2024
    """
)


In [ ]:
import threading
import time
import subprocess
import os
import sys
from pyngrok import ngrok
import requests


def kill_existing_processes():
    """
    Kill existing ngrok and Streamlit processes
    """
    try:
        # Kill ngrok tunnels
        ngrok.kill()

        # Kill existing Streamlit processes (if any)
        if sys.platform.startswith('win'):
            os.system('taskkill /F /IM streamlit.exe')
        else:
            os.system('pkill streamlit')
    except Exception as e:
        print(f"Error killing existing processes: {e}")

def check_streamlit_running():
    """
    Check if Streamlit is running on the default port
    """
    try:
        response = requests.get('http://localhost:8501')
        return response.status_code == 200
    except:
        return False

def run_app():
    """
    Run Streamlit app
    """
    try:
        subprocess.run(["streamlit", "run", "app.py"], check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error running Streamlit app: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")

def start_ngrok_tunnel():
    """
    Start ngrok tunnel and return public URL
    """
    try:
        # Ensure no existing tunnels
        ngrok.kill()

        # Connect to Streamlit's default port
        public_url = ngrok.connect(addr="8501", proto="http")
        print(f"🌐 Public URL: {public_url}")
        print(f"🔐 Public Tunnel: {ngrok.get_tunnels()[0].public_url}")

        return public_url
    except Exception as e:
        print(f"Error creating ngrok tunnel: {e}")
        return None

def main():
    kill_existing_processes()

    time.sleep(2)

    streamlit_thread = threading.Thread(target=run_app, daemon=True)
    streamlit_thread.start()

    # Chờ Streamlit khởi động
    print("🚀 Đang khởi động Streamlit...")
    time.sleep(5)

    # Kiểm tra Streamlit đã chạy chưa
    if not check_streamlit_running():
        print("❌ Streamlit không thể khởi động. Vui lòng kiểm tra app.py")
        return

    # Tạo đường hầm ngrok
    tunnel_url = start_ngrok_tunnel()

    if tunnel_url:
        print("\n🌟 Ứng dụng đã sẵn sàng!")
        print("Nhấn Ctrl+C để thoát.")

        # Giữ cho chương trình chạy
        try:
            streamlit_thread.join()
        except KeyboardInterrupt:
            print("\n🛑 Đang dừng ứng dụng...")
            ngrok.kill()

if __name__ == "__main__":
    main()

🚀 Đang khởi động Streamlit...
🌐 Public URL: NgrokTunnel: "https://15e0-34-125-110-20.ngrok-free.app" -> "http://localhost:8501"
🔐 Public Tunnel: https://15e0-34-125-110-20.ngrok-free.app

🌟 Ứng dụng đã sẵn sàng!
Nhấn Ctrl+C để thoát.
